# 원, 삼각형, 사각형 구분하기
## Local에서 작업
## 3. Data Augmentation을 사용한 CNN 모델

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(5)

Using TensorFlow backend.


### 데이터 부풀리기(Augmentation)

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [4]:
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8, 2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')
                                   
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

### 데이터셋 생성하기

In [5]:
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


### 모델 구성하기

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

### 모델 학습과정 설정하기

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [10]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

W0904 16:11:49.583984 15844 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0904 16:11:49.646978 15844 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/200
15/15 [==============================] - 4s 235ms/step - loss: 1.3395 - acc: 0.2889 - val_loss: 1.0965 - val_acc: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 10ms/step - loss: 1.1022 - acc: 0.3333 - val_loss: 1.0937 - val_acc: 0.4667
Epoch 3/200
15/15 [==============================] - 0s 10ms/step - loss: 1.0320 - acc: 0.5333 - val_loss: 1.0896 - val_acc: 0.4000
Epoch 4/200
15/15 [==============================] - 0s 10ms/step - loss: 0.8799 - acc: 0.5778 - val_loss: 1.1911 - val_acc: 0.3333
Epoch 5/200
15/15 [==============================] - 0s 10ms/step - loss: 0.7633 - acc: 0.5556 - val_loss: 1.2141 - val_acc: 0.4667
Epoch 6/200
15/15 [==============================] - 0s 10ms/step - loss: 0.7208 - acc: 0.6222 - val_loss: 2.1196 - val_acc: 0.3333
Epoch 7/200
15/15 [==============================] - 0s 11ms/step - loss: 0.6817 - acc: 0.6444 - val_loss: 1.1300 - val_acc: 0.4000
Epoch 8/200
15/15 [==============================] - 0s 16ms/step - loss: 0

15/15 [==============================] - 0s 16ms/step - loss: 0.0144 - acc: 1.0000 - val_loss: 1.5577 - val_acc: 0.8000
Epoch 63/200
15/15 [==============================] - 0s 16ms/step - loss: 0.1651 - acc: 0.9556 - val_loss: 1.2973 - val_acc: 0.8000
Epoch 64/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0354 - acc: 1.0000 - val_loss: 2.3191 - val_acc: 0.6000
Epoch 65/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0807 - acc: 0.9556 - val_loss: 1.5716 - val_acc: 0.7333
Epoch 66/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0902 - acc: 0.9778 - val_loss: 1.2142 - val_acc: 0.8000
Epoch 67/200
15/15 [==============================] - 0s 18ms/step - loss: 0.1635 - acc: 0.9333 - val_loss: 1.3618 - val_acc: 0.8000
Epoch 68/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0482 - acc: 0.9778 - val_loss: 1.1162 - val_acc: 0.8000
Epoch 69/200
15/15 [==============================] - 0s 11ms/step - loss: 0.0770 

15/15 [==============================] - 0s 22ms/step - loss: 0.1265 - acc: 0.9778 - val_loss: 2.0829 - val_acc: 0.8000
Epoch 124/200
15/15 [==============================] - 0s 22ms/step - loss: 0.0717 - acc: 0.9778 - val_loss: 2.2506 - val_acc: 0.8000
Epoch 125/200
15/15 [==============================] - 0s 21ms/step - loss: 0.1945 - acc: 0.9778 - val_loss: 2.9698 - val_acc: 0.6000
Epoch 126/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0192 - acc: 1.0000 - val_loss: 2.7403 - val_acc: 0.6000
Epoch 127/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0047 - acc: 1.0000 - val_loss: 2.7174 - val_acc: 0.6000
Epoch 128/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0286 - acc: 0.9778 - val_loss: 1.8983 - val_acc: 0.7333
Epoch 129/200
15/15 [==============================] - 0s 21ms/step - loss: 0.1580 - acc: 0.9333 - val_loss: 2.2314 - val_acc: 0.7333
Epoch 130/200
15/15 [==============================] - 0s 21ms/step - loss: 

15/15 [==============================] - 0s 26ms/step - loss: 0.0389 - acc: 0.9778 - val_loss: 2.4536 - val_acc: 0.6667
Epoch 184/200
15/15 [==============================] - 0s 26ms/step - loss: 0.0074 - acc: 1.0000 - val_loss: 3.0709 - val_acc: 0.6667
Epoch 185/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0260 - acc: 1.0000 - val_loss: 2.6954 - val_acc: 0.6667
Epoch 186/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0053 - acc: 1.0000 - val_loss: 2.4650 - val_acc: 0.7333
Epoch 187/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0037 - acc: 1.0000 - val_loss: 2.4481 - val_acc: 0.8000
Epoch 188/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0049 - acc: 1.0000 - val_loss: 2.4635 - val_acc: 0.8000
Epoch 189/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 2.4578 - val_acc: 0.8000
Epoch 190/200
15/15 [==============================] - 0s 21ms/step - loss: 

### 모델 평가하기

In [11]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 66.67%


### 모델 사용하기

In [12]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.000 0.000 1.000]
 [0.042 0.003 0.955]
 [0.000 0.000 1.000]
 [0.000 0.998 0.002]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.762 0.238]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]
 [0.007 0.336 0.657]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.950 0.000 0.050]
 [1.000 0.000 0.000]
 [0.000 0.024 0.976]]
